# FT

This notebook runs on a T4 GPU.


In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 95.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00


In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
dataset_name = "mlabonne/guanaco-llama2-1k"

dataset_old = load_dataset(dataset_name, split="train")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
dataset_old['text'][0]


'<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. </s>'

In [ ]:
def combine_text(example):
    # Remove the closing <s> from input_text and the opening <s> from target_text
    combined_text = f"{example['input_text']}  {example['target_text']} '<s>'"
    example['text'] = combined_text
    return example

# Apply the combine_text function to your dataset
combined_dataset = train_dataset.map(combine_text)

# Now each example in combined_dataset will have a new 'text' field
# which is the combination of 'input_text' and 'target_text' with
# only one <s> at the beginning and one <s> at the end

# Save the combined_dataset to a JSON file
combined_dataset.to_json('path_to_save/combined_dataset.json')


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

2422680

In [ ]:
combined_dataset['text'][0]


"<s>[INST] Given the following patent invention description: The present invention relates to a method for determining an amount of a peroxide, particularly hydrogen peroxide, by determining the luminescence of a lanthanide-ligand complex., in the context of the novelty:  The present invention relates to a method for determining an amount of a peroxide in a sample . The method comprises the steps of: —providing a sample, —contacting the sample with a terbium(III) benzene dicarboxylic acid complex, and —determining the luminescence ., what are the most relevant claims? [/INST]  </s> 1. A method for determining an amount of a peroxide in a sample, comprising the steps of: providing a sample, contacting said sample with a lanthanide-ligand complex, and determining the luminescence of said lanthanide-ligand complex, characterized in that said lanthanide-ligand complex is a terbium(III) benzene dicarboxylic acid complex. 2. The method according to claim 1, wherein said peroxide is hydrogen 

In [4]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
#dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "llama-2-7b-novelty"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
import re
from datasets import load_dataset
from transformers import pipeline

# Load summarization pipeline
summarizer = pipeline("summarization")

def summarize_abstract(abstract):
    # Summarize the abstract to extract the novelty
    summary = summarizer(abstract, max_length=75, min_length=25, do_sample=False)
    return summary[0]['summary_text']

def preprocess(example):
    description = example["description"]
    abstract = example["abstract"]
    claims = example["claims"]

    novelty = summarize_abstract(abstract)

    description_sentence = re.search(r"The present invention.*?\.", description, re.IGNORECASE)
    description = description_sentence.group(0) if description_sentence else ""

    input_text = f"<s>[INST] Given the following patent invention description: {description}, in the context of the novelty: {novelty}, what are the most relevant claims? [/INST]"

    first_claim = claims.split("\n")[0].strip()
    first_claim = re.sub(r'[\t\n\r\f\v]', ' ', first_claim)
    first_claim = re.sub(r'\s+', ' ', first_claim)
    first_claim = re.sub(r'[^\x00-\x7F]+', '', first_claim)

    target_text = f"</s> {first_claim}"

    return {"input_text": input_text, "target_text": target_text}

    import re
from datasets import load_dataset
from transformers import pipeline



# Load your dataset
dataset_dict = load_dataset('HUPD/hupd',
    name='sample',
    data_files="https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather",
    icpr_label=None,
    train_filing_start_date='2016-01-01',
    train_filing_end_date='2016-01-02',
    val_filing_start_date='2016-01-22',
    val_filing_end_date='2016-01-23',
)

# Apply the preprocess function to your dataset
train_dataset = dataset_dict["train"].map(preprocess)


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


(…)ilbart-cnn-12-6/resolve/main/config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

(…)-12-6/resolve/main/tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

(…)tilbart-cnn-12-6/resolve/main/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

(…)tilbart-cnn-12-6/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
import re
from datasets import load_dataset
from transformers import pipeline



# Load your dataset
dataset_dict = load_dataset('HUPD/hupd',
    name='sample',
    data_files="https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather",
    icpr_label=None,
    train_filing_start_date='2016-01-01',
    train_filing_end_date='2016-01-02',
    val_filing_start_date='2016-01-22',
    val_filing_end_date='2016-01-23',
)

# Apply the preprocess function to your dataset
train_dataset = dataset_dict["train"].map(preprocess)


Loading dataset with config: PatentsConfig(name='sample', version=0.0.0, data_dir='sample', data_files={'train': ['https://huggingface.co/datasets/HUPD/hupd/blob/main/hupd_metadata_2022-02-22.feather']}, description='Patent data from January 2016, for debugging')


Using metadata file: /root/.cache/huggingface/datasets/downloads/bac34b767c2799633010fa78ecd401d2eeffd62eff58abdb4db75829f8932710


Reading metadata file: /root/.cache/huggingface/datasets/downloads/bac34b767c2799633010fa78ecd401d2eeffd62eff58abdb4db75829f8932710
Filtering train dataset by filing start date: 2016-01-01
Filtering train dataset by filing end date: 2016-01-02
Filtering val dataset by filing start date: 2016-01-22
Filtering val dataset by filing end date: 2016-01-23


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

Your max_length is set to 75, but your input_length is only 39. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)
Your max_length is set to 75, but your input_length is only 32. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 75, but your input_length is only 24. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)


In [ ]:
train_dataset['input_text'][0]


'<s>[INST] Given the following patent invention description: The present invention relates to a method for determining an amount of a peroxide, particularly hydrogen peroxide, by determining the luminescence of a lanthanide-ligand complex., in the context of the novelty:  The present invention relates to a method for determining an amount of a peroxide in a sample . The method comprises the steps of: —providing a sample, —contacting the sample with a terbium(III) benzene dicarboxylic acid complex, and —determining the luminescence ., what are the most relevant claims? [/INST]'

In [ ]:

def combine_text(example):
    # Remove the closing <s> from input_text and the opening <s> from target_text
    combined_text = f"{example['input_text']}  {example['target_text']} '<s>'"
    example['text'] = combined_text
    return example

# Apply the combine_text function to your dataset
dataset = train_dataset.map(combine_text)

# Now each example in combined_dataset will have a new 'text' field
# which is the combination of 'input_text' and 'target_text'

# Save the combined_dataset to a JSON file
dataset.to_json('dataset.json')



# Apply the combine_text function to your dataset
#combined_dataset = train_dataset.map(combine_text)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

2422680

In [ ]:
dataset['text'][1]



'<s>[INST] Given the following patent invention description: The present invention relates to a method for determining an amount of a peroxide, particularly hydrogen peroxide, by determining the luminescence of a lanthanide-ligand complex., in the context of the novelty:  The present invention relates to a method for determining an amount of a peroxide in a sample . The method comprises the steps of: —providing a sample, —contacting the sample with a terbium(III) benzene dicarboxylic acid complex, and —determining the luminescence ., what are the most relevant claims? [/INST] </s> 1. A method for determining an amount of a peroxide in a sample, comprising the steps of: providing a sample, contacting said sample with a lanthanide-ligand complex, and determining the luminescence of said lanthanide-ligand complex, characterized in that said lanthanide-ligand complex is a terbium(III) benzene dicarboxylic acid complex. 2. The method according to claim 1, wherein said peroxide is hydrogen p

In [7]:
# Load dataset (you can process it here)
#dataset = load_dataset(dataset_name, split="train")
#dataset = train_dataset



# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)



(…)ma-2-7b-chat-hf/resolve/main/config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)2-7b-chat-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)b-chat-hf/resolve/main/added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

In [ ]:
# Assume the model and tokenizer have been loaded as 'model' and 'tokenizer'
# e.g., model = AutoModelForCausalLM.from_pretrained('your-model-path')
# and tokenizer = AutoTokenizer.from_pretrained('your-model-path')

# Define the description and novelty for a hypothetical patent
description = (
    "The present invention relates to a novel method of harnessing solar energy "
    "through photovoltaic cells with enhanced efficiency. The innovative design "
    "utilizes a combination of new materials and a unique arrangement of cells "
    "to maximize energy capture and minimize losses."
)
novelty = (
    "The innovative aspect of this invention is the unique arrangement of "
    "photovoltaic cells and the use of new materials that significantly enhance "
    "efficiency compared to traditional solar panels."
)

# Create the formatted prompt
prompt = f"<s>[INST] Given the following patent invention description: {description}, in the context of the novelty: {novelty}, what are the most relevant claims? [/INST]"

# Use the text-generation pipeline to generate the claims
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(prompt)

# Print the generated claims
generated_claims = result[0]['generated_text']
print(generated_claims)


<s>[INST] Given the following patent invention description: The present invention relates to a novel method of harnessing solar energy through photovoltaic cells with enhanced efficiency. The innovative design utilizes a combination of new materials and a unique arrangement of cells to maximize energy capture and minimize losses., in the context of the novelty: The innovative aspect of this invention is the unique arrangement of photovoltaic cells and the use of new materials that significantly enhance efficiency compared to traditional solar panels., what are the most relevant claims? [/INST]  Based on the provided patent invention description, the most relevant claims for the novelty of the invention are:

1. A photovoltaic cell arrangement comprising:
	* A unique combination of materials and cell arrangement to maximize energy capture and minimize losses.
	* The use of new materials that significantly enhance efficiency compared to traditional solar panels.
2. The photovoltaic cell 

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/runs

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


<s>[INST] What is a large language model? [/INST] A large language model is a type of artificial intelligence (AI) model that is trained on a large dataset of text to generate human-like language outputs. It is designed to be able to understand and generate text in a way that is similar to human language, and can be used for a wide range of applications such as chatbots, language translation, and text summarization.

Large language models are typically trained using deep learning techniques, such as recurrent neural networks (RNNs) or transformer models, and are often based on pre-trained models such as BERT or RoBERTa. These models are trained on large datasets of text, such as books, articles, or websites, and are designed to learn the patterns and structures of language.

Some examples of large language models include:

* BERT (Bidirectional Encoder Representations from Transformers


In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

20933

In [5]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

(…)ma-2-7b-chat-hf/resolve/main/config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)2-7b-chat-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)b-chat-hf/resolve/main/added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [1]:
!unzip llamaN.zip

Archive:  llamaN.zip
   creating: llama-2-7b-novelty/
  inflating: llama-2-7b-novelty/adapter_model.bin  
  inflating: llama-2-7b-novelty/README.md  
  inflating: llama-2-7b-novelty/adapter_config.json  


In [5]:
ls

dataset.json  llama-2-7b-novelty/  llamaN.zip  sample_data/


In [6]:
# Assume the model and tokenizer have been loaded as 'model' and 'tokenizer'
# e.g., model = AutoModelForCausalLM.from_pretrained('your-model-path')
# and tokenizer = AutoTokenizer.from_pretrained('your-model-path')

# Define the description and novelty for a hypothetical patent
description = (
    "The present invention relates to a novel method of harnessing solar energy "
    "through photovoltaic cells with enhanced efficiency. The innovative design "
    "utilizes a combination of new materials and a unique arrangement of cells "
    "to maximize energy capture and minimize losses."
)
novelty = (
    "The innovative aspect of this invention is the unique arrangement of "
    "photovoltaic cells and the use of new materials that significantly enhance "
    "efficiency compared to traditional solar panels."
)

# Create the formatted prompt
prompt = f"<s>[INST] Given the following patent invention description: {description}, in the context of the novelty: {novelty}, what are the most relevant claims? [/INST]"

# Use the text-generation pipeline to generate the claims
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(prompt)

# Print the generated claims
generated_claims = result[0]['generated_text']
print(generated_claims)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>[INST] Given the following patent invention description: The present invention relates to a novel method of harnessing solar energy through photovoltaic cells with enhanced efficiency. The innovative design utilizes a combination of new materials and a unique arrangement of cells to maximize energy capture and minimize losses., in the context of the novelty: The innovative aspect of this invention is the unique arrangement of photovoltaic cells and the use of new materials that significantly enhance efficiency compared to traditional solar panels., what are the most relevant claims? [/INST]  Based on the provided patent invention description, the most relevant claims for the novelty of the invention would likely be those that cover the unique arrangement of photovoltaic cells and the use of new materials. Here are some possible claims that could be relevant:

1. A method of harnessing solar energy through photovoltaic cells, comprising:
	* Arranging a plurality of photovoltaic cells

In [ ]:
!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mlabonne/llama-2-7b-miniguanaco/commit/c81a32fd0b4d39e252326e639d63e75aa68c9a4a', commit_message='Upload tokenizer', commit_description='', oid='c81a32fd0b4d39e252326e639d63e75aa68c9a4a', pr_url=None, pr_revision=None, pr_num=None)